# Using a pretrained model

In [1]:
# imports
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.utils import image_dataset_from_directory
import pathlib
import numpy as np
import matplotlib.pyplot as plt

c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\users\danie\pycharmprojects\tensorflowtest\venv\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# data augmenting layer
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2)
])

conv_base = keras.applications.vgg16.VGG16(weights="imagenet", include_top=False, input_shape=(180, 180, 3))

In [5]:
# preprocessing data
base_dir = pathlib.Path("dogs-vs-cats/small")
train_dataset = image_dataset_from_directory(base_dir / "train",
                                            image_size=(180, 180), batch_size=32)
validation_dataset = image_dataset_from_directory(base_dir / "validation",
                                            image_size=(180, 180), batch_size = 32)
test_dataset = image_dataset_from_directory(base_dir / "test",
                                            image_size = (180, 180), batch_size=32)

Found 2000 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [6]:
def features_and_labels(dataset):
    all_features = []
    all_labels = []
    for image_batch, label_batch in dataset:
        processed_images = keras.applications.vgg16.preprocess_input(image_batch)
        features = conv_base(processed_images)
        all_features.append(features)
        all_labels.append(label_batch)
    return np.concatenate(all_features), np.concatenate(all_labels)

train_features, train_labels = features_and_labels(train_dataset)
val_features, val_labels = features_and_labels(validation_dataset)
test_features, test_labels = features_and_labels(test_dataset)

KeyboardInterrupt: 

In [8]:
inputs = keras.Input(shape=(5, 5, 512))
x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

callbacks = [keras.callbacks.ModelCheckpoint(filepath="feature_extraction.keras", save_best_only=True, monitor="val_loss")]
history = model.fit(train_features, train_labels, epochs=20,
                    validation_data=(val_features, val_labels), callbacks=callbacks)

acc = history.history["accuracy"]
loss = history.history["loss"]
val_acc = history.history["val_accuracy"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Training Accuracy")
plt.plot(epochs, val_acc, "b", label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training Loss")
plt.plot(epochs, val_acc, "b", label="Validation Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

In [11]:
test_model = keras.models.load_model("feature_extraction.keras")
test_loss, test_acc = test_model.evaluate(test_features, test_labels)
print(f"Test Accuracy: {test_acc:.3f}")


NameError: name 'test_features' is not defined